In [4]:
###### Finger angle using MediaPipe for one hand

import cv2
import mediapipe as mp
import math
import csv

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize VideoCapture with the webcam
cap = cv2.VideoCapture(0)

# Create a CSV file for storing angle information
csv_file_path = 'hand_angles.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header to the CSV file
    csv_writer.writerow(['Frame', 'Thumb MCP-PIP', 'Index MCP-PIP', 'Middle MCP-PIP', 'Ring MCP-PIP', 'Pinky MCP-PIP'])

    frame_count = 0

    # Set the color for landmarks and lines (BGR format)
    point_color = (0, 255, 0)  # 
    line_color = (3, 53, 0)   # 
    letter_color = (3,0,0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        rgb_frame = cv2.flip(rgb_frame, 1)

        # Set flag
        rgb_frame.flags.writeable = False

        # Detections
        results = hands.process(rgb_frame)

        # Set flag to true
        rgb_frame.flags.writeable = True

        # RGB to BGR
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Angle calculation
        hand_landmarks = results.multi_hand_landmarks
        if hand_landmarks:
            hand_landmarks = hand_landmarks[0].landmark  # Assuming only one hand is detected
            # Define the points for calculating angles
            points = [hand_landmarks[0],  # Thumb MCP
                      hand_landmarks[2],  # Thumb PIP
                      hand_landmarks[3],  # Thumb DIP
                      hand_landmarks[5],  # Index MCP
                      hand_landmarks[6],  # Index PIP
                      hand_landmarks[7],  # Index DIP
                      hand_landmarks[9],  # Middle MCP
                      hand_landmarks[10], # Middle PIP
                      hand_landmarks[11], # Middle DIP
                      hand_landmarks[13], # Ring MCP
                      hand_landmarks[14], # Ring PIP
                      hand_landmarks[15], # Ring DIP
                      hand_landmarks[17], # Pinky MCP
                      hand_landmarks[18], # Pinky PIP
                      hand_landmarks[19]] # Pinky DIP

            # Calculate angles
            angles = [math.degrees(math.atan2(points[i + 1].y - points[i].y, points[i + 1].x - points[i].x)) -
                      math.degrees(math.atan2(points[i + 2].y - points[i + 1].y, points[i + 2].x - points[i + 1].x))
                      for i in range(0, len(points), 3)]

            # Display angles on the frame with user-specified point color
            for i, angle in enumerate(angles):
                cv2.putText(frame, f'{i + 1}: {angle:.2f} deg', (10, 30 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            letter_color, 1, cv2.LINE_AA)

            # Draw landmarks and lines with user-specified colors
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=line_color, thickness=3, circle_radius=3),
                                      mp_drawing.DrawingSpec(color=point_color, thickness=4, circle_radius=4))

            # Write angle information to CSV file
            csv_writer.writerow([frame_count] + angles)

        # Display the frame
        cv2.imshow('Hand Tracking', frame)

        # Increment frame count
        frame_count += 1

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

print(f'Angle information saved to {csv_file_path}')



Angle information saved to hand_angles.csv


In [5]:
###### Finger blending angles for one hand

import cv2
import mediapipe as mp
import math
import csv

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize VideoCapture with the webcam
cap = cv2.VideoCapture(0)

# Create a CSV file for storing finger bending information
csv_file_path = 'finger_bending.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header to the CSV file
    csv_writer.writerow(['Frame', 'Thumb Bending', 'Index Bending', 'Middle Bending', 'Ring Bending', 'Pinky Bending'])

    frame_count = 0

    # Set the color for hand landmarks (BGR format)
    point_color = (0, 255, 0)  # Green

    # Set the color for hand connecting lines (BGR format)
    line_color = (255, 0, 0)   # Blue

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        rgb_frame = cv2.flip(rgb_frame, 1)

        # Set flag
        rgb_frame.flags.writeable = False

        # Detections
        results = hands.process(rgb_frame)

        # Set flag to true
        rgb_frame.flags.writeable = True

        # RGB to BGR
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Finger bending calculation
        hand_landmarks = results.multi_hand_landmarks
        if hand_landmarks:
            hand_landmarks = hand_landmarks[0].landmark  # Assuming only one hand is detected

            # Define the finger joints for bending calculation
            finger_joints = {
                'thumb': [0, 1, 2],
                'index': [0, 5, 6, 7],
                'middle': [0, 9, 10, 11],
                'ring': [0, 13, 14, 15],
                'pinky': [0, 17, 18, 19]
            }

            # Calculate finger bending angles
            finger_bendings = {}
            for finger, joints in finger_joints.items():
                angles = [math.degrees(math.atan2(hand_landmarks[joints[i + 1]].y - hand_landmarks[joints[i]].y,
                                                  hand_landmarks[joints[i + 1]].x - hand_landmarks[joints[i]].x))
                          for i in range(len(joints) - 1)]
                finger_bendings[finger] = angles

            # Display finger bending angles on the frame with custom colors
            # Display finger bending angles on the frame
            y_offset = 30
            for finger, angles in finger_bendings.items():
                display_text = f'{finger.capitalize()} Bending:'
                for angle in angles:
                    display_text += f' {angle:.2f} deg'
                cv2.putText(frame, display_text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                y_offset += 20

            # Draw hand landmarks with custom point color
            for landmark in hand_landmarks:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 4, point_color, -1)  # -1 fills the circle

            # Draw hand connecting lines with custom line color
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=line_color, thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=point_color, thickness=2, circle_radius=4))




            # Write finger bending information to CSV file
            csv_writer.writerow([frame_count] + [angle for angles in finger_bendings.values() for angle in angles])

        # Display the frame
        cv2.imshow('Hand Tracking', frame)

        # Increment frame count
        frame_count += 1

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

print(f'Finger bending information saved to {csv_file_path}')


Finger bending information saved to finger_bending.csv


In [12]:
####### Face points detection and storing the data

import cv2
import mediapipe as mp
import csv

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Initialize VideoCapture with the webcam
cap = cv2.VideoCapture(0)

# Create a CSV file for storing face mesh landmark coordinates
csv_file_path = 'face_mesh_landmarks.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Write header to the CSV file
    header = ['Frame']
    for i in range(468):  # 468 landmarks in total
        header.extend([f'X_{i}', f'Y_{i}', f'Z_{i}'])
    csv_writer.writerow(header)

    frame_count = 0

    # Set the color for face mesh landmarks (BGR format)
    point_color = (0, 255, 0)  # Green

    # Set the size of the face mesh landmarks
    point_size = 1

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Set flag
        rgb_frame.flags.writeable = False

        # Detections
        results = face_mesh.process(rgb_frame)

        # Set flag to true
        rgb_frame.flags.writeable = True

        # RGB to BGR
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Face mesh landmarks
        if results.multi_face_landmarks:
            landmarks_data = [frame_count]

            for face_landmarks in results.multi_face_landmarks:
                for i, landmark in enumerate(face_landmarks.landmark):
                    cx, cy = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    cz = landmark.z  # Depth (Z-coordinate)
                    cv2.circle(frame, (cx, cy), point_size, point_color, -1)  # Custom color and size for landmarks

                    landmarks_data.extend([landmark.x, landmark.y, landmark.z])

            # Write face mesh landmark coordinates to CSV file
            csv_writer.writerow(landmarks_data)

        # Display the frame
        cv2.imshow('Face Mesh', frame)

        # Increment frame count
        frame_count += 1

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

print(f'Face mesh landmark coordinates saved to {csv_file_path}')


Face mesh landmark coordinates saved to face_mesh_landmarks.csv


In [23]:
##### Pupil detection and stroing the points

import cv2
import mediapipe as mp
import csv
import numpy as np

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Initialize VideoCapture with the webcam
cap = cv2.VideoCapture(0)

# Define the indices for the landmarks associated with the right eye
right_eye_indices = [33, 246, 161, 160, 159, 158, 157, 173, 133, 7, 163, 144, 145, 153, 154, 155]

# Define the indices for the landmarks associated with the left eye
left_eye_indices = [362, 398, 384, 385, 386, 387, 388, 466, 263, 249, 390, 373, 374, 380, 381, 382]

# Create a CSV file for storing iris coordinates
csv_file_path = 'iris_coordinates.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write header to the CSV file
    csv_writer.writerow(['Frame'] + [f'right_{index}_x' for index in right_eye_indices] + [f'right_{index}_y' for index in right_eye_indices] +
                       [f'left_{index}_x' for index in left_eye_indices] + [f'left_{index}_y' for index in left_eye_indices] +
                       ['right_average_x', 'right_average_y', 'left_average_x', 'left_average_y'])

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Set flag
        rgb_frame.flags.writeable = False

        # Detections
        results = face_mesh.process(rgb_frame)

        # Set flag to true
        rgb_frame.flags.writeable = True

        # RGB to BGR
        frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        # Landmark coordinates
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                right_landmark_coords = []
                left_landmark_coords = []

                # Extract coordinates for specified landmarks for the right eye
                for index in right_eye_indices:
                    landmark = face_landmarks.landmark[index]
                    x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    right_landmark_coords.extend([x, y])

                    # Display landmarks on the frame
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

                # Extract coordinates for specified landmarks for the left eye
                for index in left_eye_indices:
                    landmark = face_landmarks.landmark[index]
                    x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    left_landmark_coords.extend([x, y])

                    # Display landmarks on the frame
                    cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

                # Calculate average coordinates for each eye
                right_avg_x = int(np.mean(right_landmark_coords[::2]))
                right_avg_y = int(np.mean(right_landmark_coords[1::2]))

                left_avg_x = int(np.mean(left_landmark_coords[::2]))
                left_avg_y = int(np.mean(left_landmark_coords[1::2]))

                # Display average points on the frame in blue (right) and orange (left)
                cv2.circle(frame, (right_avg_x, right_avg_y), 5, (255, 0, 0), -1)
                cv2.circle(frame, (left_avg_x, left_avg_y), 5, (0, 165, 255), -1)

                # Write landmark coordinates and average coordinates to CSV file
                csv_writer.writerow([frame_count] + right_landmark_coords + left_landmark_coords +
                                   [right_avg_x, right_avg_y, left_avg_x, left_avg_y])

        # Display the frame
        cv2.imshow('Landmark Tracking', frame)

        # Increment frame count
        frame_count += 1

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the VideoCapture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

print(f'Landmark coordinates saved to {csv_file_path}')


Landmark coordinates saved to iris_coordinates.csv
